<a href="https://colab.research.google.com/github/MarcosRolando/OrgaDeDatosTP2/blob/main/neuralNetwoork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import feature_column


In [2]:
df = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')
df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) #0 corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won. Asi tenemos un problema de clasificacion binario que puede entender la red neuronal.

#Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
#TODO: Analizar si el Sales_Contract_No no es que importe el numero en si, sino si tiene
#o no tiene numero de contrato. Por ahora no lo meto como input.
#TODO: Ver el mismo tema con la columna 'Price', la mayoria tiene None u Other
#y solo unos pocos tienen precio numerico. Quiza importe que tenga precio o no tenga,
#o si no tiene precio quiza importe si es None u Other. Por ahora no lo pongo
#como input.
df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                 'Prod_Category_A', 'ID', 'Opportunity_ID', 'Sales_Contract_No',
                 'Price'],inplace=True)

#Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
#Estos particularmente son whitespace, coma y parentesis por ejemplo.
df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                   'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                   'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

#Columnas que consideramos numericas
numeric_columns = ['Pricing_Delivery_Terms_Quote_Appr','Pricing_Delivery_Terms_Approved',
                     'Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved',
                     'ASP','ASP_converted','TRF','Total_Amount','Total_Taxable_Amount']

#Columnas que consideramos categoricas de pocos valores posibles
indicator_columns = ['Region','Territory','Bureaucratic_Code','Source',
                       'Billing_Country','Account_Name','Opportunity_Name',
                       'Account_Owner','Opportunity_Owner','Account_Type',
                       'Opportunity_Type','Quote_Type','Delivery_Terms',
                       'Brand','Product_Type','Size','Product_Category_B',
                       'Currency','Last_Modified_By','Product_Family',
                       'Product_Name','ASP_Currency','Total_Amount_Currency',
                       'Total_Taxable_Amount_Currency']

#Drope columnas que vamos a usar pero por ahora no las uso
df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                 'Quote_Expiry_Date','Last_Modified_Date',
                 'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                 'Month','Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date'],inplace=True)

df.columns

#Debemos separar algunos de los registros para armar un set de test propio (no el de la catedra). De esta forma sabremos rapidamente
#si nuestro modelo esta dando resultados optimos o no sin necesidad de estar subiendo el TP a Kaggle constantemente.
#Sin embargo, no queremos usar tantos registros ya que estariamos disminuyendo el set de entrenamiento considerablemente.
#Podemos empezar reservando 2000 registros para el test de prueba y ver que onda. Pasariamos de tener 16 mil a 14 mil 
#registros para el set de entrenamiento, no es una perdida importantisima creo en principio, asi que arrancamos con eso.

#Por otro lado, nuestro test de prueba deberia tener un 50 50 de Closed Won y Closed Lost, por lo que no podemos elegir asi nomas
#al azar.

#TODO: DEJO COMO TAREA ARMAR EL SET DE PRUEBA, POR AHORA SOLO NOS PREOCUPAMOS DE LOGRAR HACER ANDAR LA RED NEURONAL.

Index(['Region', 'Territory', 'Pricing_Delivery_Terms_Quote_Appr',
       'Pricing_Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Bureaucratic_Code', 'Source',
       'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Account_Owner',
       'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Quote_Type',
       'Delivery_Terms', 'Brand', 'Product_Type', 'Size', 'Product_Category_B',
       'Currency', 'Last_Modified_By', 'Product_Family', 'Product_Name',
       'ASP_Currency', 'ASP', 'ASP_converted', 'TRF', 'Total_Amount_Currency',
       'Total_Amount', 'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount',
       'Stage'],
      dtype='object')

In [3]:
# Metodo que pasa de DataFrame de Pandas a un DataSet de TensorFlow
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Stage') #Retorna la columna Stage, eliminandolo simultaneamente del DataFrame. 'Stage' seria nuestra columna 'target', es decir, lo que queremos predecir.
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)) #Crea un DataSet cuyos elementos son slices de los tensores pasados a la funcion. Ver documentacion oficial para comprender bien.
  #En pocas palabras, le pasamos las columnas con los datos como diccionario (estilo 'columna':[dato1,dato2,dato3]) y una lista con los resultados estilo [resu1,resu2,resu3].
  #Se genera un DataSet del estilo [('columna':[dato1,dato2,dato3], [resu1, resu2, resu3])].
  #En realidad 'columna' es una lista de todas las columnas con sus correspondientes datos, pero se entiende la idea creo.
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe)) #Al tener el buffer_size del mismo tamanio que la cantidad de datos del dataset, tenemos perfect shuffling (ver documentacion para comprender).
    #Basicamente mezlcamos el dataset para que luego los batches que se armen contengan distintos elementos si lo entrenamos distintas veces.
  ds = ds.batch(batch_size) #Arma batches de tamanio batch_size entre elementos consecutivos del DataSet.
  return ds

In [4]:
#Arma las features, asignando las columnas del DataFrame segun corresponda al tipo de feature
#(numerico, categorico, etc). Entiendase por feature a las columnas del DataFrame.
def set_up_feature_columns(dataframe, numeric_columns, indicator_columns):
  features = []

  #numeric features
  for column_name in numeric_columns:
    features.append(feature_column.numeric_column(column_name))

  # indicator_columns (one-hot value vector, para aquellas columnas categoricas de pocas opciones)
  for column_name in indicator_columns:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
                                          column_name, dataframe[column_name].unique())
    indicator_column = feature_column.embedding_column(categorical_column, dimension=8)
    features.append(indicator_column)

  return features

In [5]:
  features = set_up_feature_columns(df,numeric_columns,indicator_columns)
  feature_layer = tf.keras.layers.DenseFeatures(features)
  ds = df_to_dataset(df,batch_size=32)

In [6]:
model = tf.keras.Sequential([
  feature_layer,
  tf.keras.layers.Dense(3000, activation='sigmoid'),
  tf.keras.layers.Dense(1000, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

#Ignoren el WARNING, esta en la documentacion tambien. Nadie le da bola en StackOverflow xd.
model.fit(ds,epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
528/528 [==============================] - 24s 34ms/step - loss: 2.0075 - accuracy: 0.4396
Epoch 2/10
528/528 [==============================] - 18s 34ms/step - loss: 1.1534 - accuracy: 0.4466
Epoch 3/10
528/528 [==============================] - 18s 34ms/step - loss: 0.9658 - accuracy: 0.4334
Epoch 4/10
528/528 [==============================] - 18s 34ms/step - loss: 0.8459 - accuracy: 0.4354
Epoch 5/10
528/528 [==============================] - 18s 34ms/step - loss: 0.7727 - accuracy: 0.4360
Epoch 6/10
528/528 [==============================] - 18s 35ms/step - loss: 0.7301 - accuracy: 0.4304
Epoch 7/10
528/528 [==============================] - 19s 36ms/step - loss: 0.7031 - accuracy: 0.4345
Epoch 8/10
528/528 [==============================] - 20s 37ms/step - loss: 0.6905 - accuracy: 0.5715
Epoch 9/10
528/528 [==============================] - 19s 36ms/step - loss

In [7]:
loss, accuracy = model.evaluate(ds)
print("Accuracy", accuracy)

Consider rewriting this model with the Functional API.
528/528 [==============================] - 15s 24ms/step - loss: 0.6848 - accuracy: 0.5647
Accuracy 0.56465083360672
